In [1]:
from model import atp_graph
from data import synthetic_data_gen, feature_extractor
import keras
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import tensorflow as tf
import IPython
import time
import os

from model import atp_pipeline

2023-04-28 13:48:00.017358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
t, Y = synthetic_data_gen.GP_functions_gen(samples=200)

In [3]:
t = t[:,:,np.newaxis]


In [4]:
Y.shape

(200, 100, 1)

In [5]:
save_dir = "/users/rapart57/rapart57-hpc/ATP/weights/gp/rbf"

In [6]:
tf.random.set_seed(442)
opt = tf.keras.optimizers.Adam(3e-4)
run=2; heads=2
atp_model = atp_pipeline.atp_pipeline()

EPOCHS = 40



2023-04-28 13:48:04.637464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 13:48:05.333456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6975 MB memory:  -> device: 0, name: Quadro P4000, pci bus id: 0000:3b:00.0, compute capability: 6.1


In [7]:
seq_l = Y.shape[1];
num_batches = 500
batch_s=20
mini = 50000
n_C_te =5

In [27]:
step = 1
tr_step, te_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [26]:
    for epoch in range(2):
        print("epoch",+epoch)
        start = time.time()
        for batch_n in range(10):

            tr_ix = np.random.choice(np.arange(0, int(0.9 * Y.shape[0])), batch_s)
            n_C = int(np.random.choice(np.arange(2, 9), 1))
            n_T = 100 - n_C
            y_tr = Y[tr_ix]
            t_tr = t[tr_ix]
                 
            _,_, nll_pp, msex = tr_step(atp_model, opt, t_tr,y_tr,n_C,n_T)

            if ((batch_n % 2 == 0)):
                te_ix = np.random.choice(np.arange(int(0.9 * Y.shape[0]), Y.shape[0]), batch_s)
                y_te = Y[te_ix]
                t_te = t[te_ix]
                
                _,_, nll_pp_te, msex_te = te_step(atp_model, t_tr,y_tr,n_C,n_T)
                
            if nll_pp_te < mini:
                mini = nll_pp_te
                IPython.display.clear_output(wait=True)
                print("epoch {} batch {} test lik pp: {}".format(epoch, batch_n, nll_pp)) 
                manager.save()
                step += 1
                ckpt.step.assign_add(1)
        print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

epoch 0 batch 4 test lik pp: -0.6643770337104797
Time taken for 1 epoch: 0.6566910743713379 secs

epoch 1
Time taken for 1 epoch: 0.3406245708465576 secs



In [29]:
atp_model([t_tr,y_tr,n_C,n_T,False])

(<tf.Tensor: shape=(20, 98, 1), dtype=float32, numpy=
 array([[[-1.1341854 ],
         [-1.7527193 ],
         [-0.9662785 ],
         ...,
         [-2.2222342 ],
         [-0.61828864],
         [ 0.17751393]],
 
        [[-1.737303  ],
         [-1.473785  ],
         [-1.3551419 ],
         ...,
         [-0.46409088],
         [-1.3993181 ],
         [-1.8215457 ]],
 
        [[-0.6228004 ],
         [-0.5951429 ],
         [-0.93868095],
         ...,
         [-1.4971751 ],
         [-1.9156438 ],
         [-1.6816232 ]],
 
        ...,
 
        [[-1.0192115 ],
         [-1.3992127 ],
         [-1.0287672 ],
         ...,
         [-1.9469209 ],
         [-1.0841346 ],
         [-1.4068415 ]],
 
        [[-1.6027719 ],
         [-1.442345  ],
         [-1.531571  ],
         ...,
         [-1.4928826 ],
         [-1.0883497 ],
         [-1.4992023 ]],
 
        [[-2.4462092 ],
         [-1.1879479 ],
         [-2.3719404 ],
         ...,
         [-0.103708  ],
         [-0.040

In [30]:
atp_model_new = atp_pipeline.atp_pipeline()

In [31]:
step = 1
tr_step, te_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model_new)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [32]:
atp_model([t_tr,y_tr,n_C,n_T,False])

(<tf.Tensor: shape=(20, 98, 1), dtype=float32, numpy=
 array([[[-1.1341854 ],
         [-1.7527193 ],
         [-0.9662785 ],
         ...,
         [-2.2222342 ],
         [-0.61828864],
         [ 0.17751393]],
 
        [[-1.737303  ],
         [-1.473785  ],
         [-1.3551419 ],
         ...,
         [-0.46409088],
         [-1.3993181 ],
         [-1.8215457 ]],
 
        [[-0.6228004 ],
         [-0.5951429 ],
         [-0.93868095],
         ...,
         [-1.4971751 ],
         [-1.9156438 ],
         [-1.6816232 ]],
 
        ...,
 
        [[-1.0192115 ],
         [-1.3992127 ],
         [-1.0287672 ],
         ...,
         [-1.9469209 ],
         [-1.0841346 ],
         [-1.4068415 ]],
 
        [[-1.6027719 ],
         [-1.442345  ],
         [-1.531571  ],
         ...,
         [-1.4928826 ],
         [-1.0883497 ],
         [-1.4992023 ]],
 
        [[-2.4462092 ],
         [-1.1879479 ],
         [-2.3719404 ],
         ...,
         [-0.103708  ],
         [-0.040

In [9]:
import math as m

In [10]:
n_C = int(np.random.choice(np.arange(2, 9), 1))
n_T = 100 - n_C
                 

In [11]:

            tr_ix = np.random.choice(np.arange(0, int(0.9 * Y.shape[0])), batch_s)
            n_C = int(np.random.choice(np.arange(2, 9), 1))
            n_T = 100 - n_C
            y_tr = Y[tr_ix, :, :]
            t_tr = t[tr_ix, :,:]
                 

In [13]:
x = t[:,:n_C+n_T,:]
y = Y[:,:n_C+n_T,:]

In [14]:
from data import feature_extractor

In [15]:
   def PE(inputs):  # return.shape=(T,B,d)
        # t.shape=(T,B)   T=sequence_length, B=batch_size
        """A position-embedder, similar to the Attention paper, but tweaked to account for
        floating point positions, rather than integer.

        """

        x, enc_dim, xΔmin, xmax = inputs

        R = xmax / xΔmin * 100
        drange_even = tf.cast(xΔmin * R**(tf.range(0,enc_dim,2)/enc_dim),"float32")
        drange_odd = tf.cast(xΔmin * R**((tf.range(1,enc_dim,2) - 1)/enc_dim),"float32")
        x = tf.concat([tf.math.sin(x / drange_even), tf.math.cos(x / drange_odd)], axis=2)
        return x   

In [16]:
x_emb = [PE([x[:,:,i][:,:,tf.newaxis],32,0.1,2]) for i in range(x.shape[-1])] 
x_emb = tf.concat(x_emb,axis=-1)

In [17]:
from data import feature_extractor

In [18]:
y_diff,x_diff,d,x_n,y_n = feature_extractor.DE()([y,x,n_C,n_T,True])


else loop
(200, 100, 1)


In [19]:
d.shape

TensorShape([200, 100, 1])

In [50]:
        inputs_for_processing = [x_emb,y,y_diff,x_diff,d,x_n,y_n,n_C,n_T]
